# BPR: Bayesian Personalized Ranking from Implicit Feedback

****

## 目录

> <div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><li><span><a href="#Algorithm" data-toc-modified-id="Algorithm-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Algorithm</a></span><ul class="toc-item"><li><span><a href="#model" data-toc-modified-id="model-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>model</a></span></li><li><span><a href="#Optimization" data-toc-modified-id="Optimization-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Optimization</a></span></li><li><span><a href="#Learning-algorithm" data-toc-modified-id="Learning algorithm-2.1"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Learning algorithm</a></span></li></ul></li><li><span><a href="#Reference" data-toc-modified-id="Reference"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reference</a></span></ul></div>

## Introduction

    当物品数量非常多时，对用户推荐几个其最感兴趣的物品是推荐系统中一个典型的问题(Top-N)。
    在有些推荐场景中，需要在千万级别的商品中推荐个位数的商品给用户，而此时，我们更关心的是，哪些极少数商品在用户心中有更高的优先级，也就是排序更靠前。因此，我们需要一个排序算法，这个算法可以把每个用户对应的所有商品按喜好排序。BPR就是这样的一个我们需要的排序算法。

## Algorithm

### model

$我们发现有任意用户u，其同时观测到了物品i和物品j，但只对物品i进行了反馈，$
>$我们把这种现象记作i>_{u}j,称作用户u对物品i基于物品j的偏好,以下简称为用户u对物品i的偏好$

>$我们把用户u对其观测到的物品i的偏好进行的排序，称作偏序$

$其次，为了建立贝叶斯模型，我们作如下假设:$
> $\textbf{任意用户u的偏好与其他用户的偏好相互独立}，即用户u对任意同时观测到的物品i,j的偏好与其他用户的偏好无关$

> $\textbf{同一用户u对不同物品i的偏序相互独立}，即用户u对任意同时观测到的物品i,j的偏序和他对其他商品的偏序无关$

$根据以上性质，我们发现，这个排序关系符号>_u满足完整性，反对称性和传递性，即在用户集U和物品集I中：$
>$完整性：\forall{i,j}\in I: i \neq j \Rightarrow i >_u j \cup j >_u i$

>$反对称性：\forall{i,j}\in I: i >_u j \cap j >_u i \Rightarrow i = j$

>$传递性：\forall{i,j，k}\in I: i >_u j \cap j >_u k \Rightarrow i >_u k$

$假定用户总数为m,物品总数为n,我们将上述的用户对物品的反馈量化为矩阵的形式：$

user\ item | $i_1$ | $i_2$ | ... | $i_n$
:-: | :-: | :-: | :-: | :-:
 $u_1$ | 1 | 0 | ... | 0| 
 $u_2$ | 0 | 1 | ... | 1|
 ...|       
 $u_m$ | 0 | 0 | ...| 1|

$我们将该矩阵记作X，称作评估矩阵，可以看到其中用户有反馈的数值为1，无反馈的数值为0。$

$对X进行矩阵分解，即X可以被分解为两个低维矩阵，因此我们需要定义如下：$

> $定义m行k列的矩阵，记作W，称作用户特征矩阵$

> $定义n行k列的矩阵，记作H，称作物品特征矩阵$

> $\overline{X} = WH^T$

$所以对于任意一个用户u，对应的任意一个物品i我们期望有：$
\begin{align}
\overline{x}_{ui} = w_{u} \cdot h_{i} = \sum\limits_{f = 1}^k w_{uf}h_{if}
\end{align}

$因此，我们的目标为优化矩阵W,H，使得\overline{X}和X最相似。$

### Optimization

$我们有三元数据集如下：$
\begin{align}
D_S := \{(u,i,j)|i为用户观测到的物品,j \in I \cap \{j| i >_u j\}\}
\end{align}

$我们把用户u对所有物品的排序记作>_u，则有最大后验概率估计P(W,H|>_u)，我们把参数W,H用\theta来代替:$

\begin{align}
P(W,H|>_u) \Rightarrow P(\theta|>_u)
\end{align}

$根据贝叶斯公式，我们有：$

\begin{align}
P(\theta|>_u) = \frac{P(>_u|\theta)P(\theta)}{P(>_u)}
\end{align}

$根据先前假设，用户u的偏好与其他用户的偏好相互独立,因此P(>_u)对不同物品而言是一个固定值,所以:$

\begin{align}
P(\theta|>_u) \propto P(>_u|\theta)P(\theta)
\end{align}

\begin{align}
\hat{\theta} = arg \mathop {\max }\limits_{\theta} P(\theta|>_u)  \Leftrightarrow arg \mathop {\max }\limits_{\theta}{P(>_u|\theta)P(\theta)} \Leftrightarrow arg \mathop {\max }\limits_{\theta} ln({P(>_u|\theta)P(\theta)})
\end{align}

$这个优化目标转化为两部分。第一部分和样本数据集D有关，第二部分和样本数据集D无关。$

$对于第一部分，由于我们假设每个用户之间的偏好行为相互独立，同一用户对不同物品的偏序相互独立，所以有：$

\begin{align}
\prod_{u \in U}P(>_u|\theta) = \prod_{(u,i,j) \in (U \times I \times I)}P(i >_u j|\theta)^{\delta((u,i,j) \in D)}(1-P(i >_u j|\theta))^{\delta((u,j,i) \not\in D) }
\end{align}

$其中:$

\begin{align}
\delta(b)= \begin{cases} 1& {if\; b\; is \;true}\\ 0& {else} \end{cases}
\end{align}

$根据上面讲到的完整性和反对称性，优化目标的第一部分可以简化为：$

\begin{align}
\prod_{u \in U}P(>_u|\theta) = \prod_{(u,i,j) \in D}P(i >_u j|\theta)
\end{align}

$而对于P(i>uj|θ)这个概率，我们可以使用下面这个式子来代替:$

\begin{align}
P(i >_u j|\theta) = \sigma(\overline{x}_{uij}(\theta))
\end{align}

$其中\sigma(x)为sigmoid函数，即:$

\begin{align}
\sigma(x) = \frac{1}{1+e^{-x}}
\end{align}

$对于\overline{x}_{uij}(\theta), 我们想要使其满足当i >_u j， \overline{x}_{uij}(\theta)>0, 而当j >_u i， \overline{x}_{uij}(\theta) <0，我们使用如下形式来表示:$

\begin{align}
\overline{x}_{uij}(\theta) = \overline{x}_{ui}(\theta) - \overline{x}_{uj}(\theta)
\end{align}

$其中，\overline{x}_{ui}(\theta)，\overline{x}_{uj}(\theta)表示\overline{X}中对应的值, 简便起见，我们省去式子中的\theta，则有：$

\begin{align}
\overline{x}_{uij} = \overline{x}_{ui} - \overline{x}_{uj}
\end{align}

$因此第一部分最终形式为:$

\begin{align}
\prod_{u \in U}P(>_u|\theta) = \prod_{(u,i,j) \in D}P(i >_u j|\theta)=\prod_{(u,i,j) \in D}\sigma(\overline{x}_{ui} - \overline{x}_{uj})
\end{align}

$对于第二部分P(\theta)，使用贝叶斯假设，即这个概率分布符合正太分布，且均值为0，协方差矩阵是\lambda_{\theta}I,即：$

\begin{align}
P(\theta) \sim N(0, \lambda_{\theta}I)
\end{align}

$因此第二部分的最终形式为:$

\begin{align}
P(\theta) = \frac{1}{\sqrt{2\pi\lambda_{\theta}I}}e^{\frac{-\theta^2}{2(\lambda_{\theta}I)}}
\end{align}

$我们对后验概率估计取对数得到的最终形式为:$

\begin{align}
ln\;P(\theta|>_u) \propto ln\;P(>_u|\theta)P(\theta) &= ln\;\prod\limits_{(u,i,j) \in D} \sigma(\overline{x}_{ui} - \overline{x}_{uj}) + ln P(\theta) \\
&= \sum\limits_{(u,i,j) \in D}ln\sigma(\overline{x}_{ui} - \overline{x}_{uj}) + \lambda||\theta||^2\;
\end{align}

$在这里我们使用梯度上升(下降)法，对\theta求导:$

\begin{align}
\frac{\partial ln\;P(\theta|>_u)}{\partial \theta} \propto \sum\limits_{(u,i,j) \in D} \frac{1}{1+e^{\overline{x}_{ui} - \overline{x}_{uj}}}\frac{\partial (\overline{x}_{ui} - \overline{x}_{uj})}{\partial \theta} + \lambda \theta
\end{align}

$其中:$

\begin{align}
\overline{x}_{ui} - \overline{x}_{uj} = \sum\limits_{f=1}^kw_{uf}h_{if} - \sum\limits_{f=1}^kw_{uf}h_{jf}
\end{align}

$这样我们可以求出：$

\begin{align}
\frac{\partial (\overline{x}_{ui} - \overline{x}_{uj})}{\partial \theta} = \begin{cases} (h_{if}-h_{jf})& {if\; \theta = w_{uf}}\\ w_{uf}& {if\;\theta = h_{if}} \\ -w_{uf}& {if\;\theta = h_{jf}}\end{cases}
\end{align}

$由此式，我们可以对其中的参数进行迭代更新，从而取得概率最大的有效收敛值对应的参数。$

### Learning algorithm

$输入: 已有数据集D,学习速率\alpha, 正则化参数\lambda, 特征值维度k$

$输出: 模型参数，即矩阵W,H$

$步骤：$
> $1. 初始化W,H$

> $2. 迭代更新参数，公式如下：$
\begin{align}
w_{uf} &= w_{uf} + \alpha(\sum\limits_{(u,i,j)\in D}\frac{1}{1+e^{\overline{x}_{ui} - \overline{x}_{uj}}}(h_{if}-h_{jf})+\lambda w_{uf})\\
h_{if} &= h_{if} + \alpha(\sum\limits_{(u,i,j)\in D}\frac{1}{1+e^{\overline{x}_{ui} - \overline{x}_{uj}}}w_{uf} + \lambda h_{if})\\
h_{jf} &= h_{jf} + \sum\limits_{(u,i,j)\in D}\frac{1}{1+e^{\overline{x}_{ui} - \overline{x}_{uj}}}(-w_{uf}) + \lambda h_{jf})
\end{align}

> $3.重复步骤2，直到W,H收敛，输出W,H$

> $4.通过W,H计算每个用户对物品的偏好值，根据偏好值对物品进行排序，最终给出排名最高的n个物品$

## Reference

   1. [BPR: Bayesian Personalized Ranking from Implicit Feedback](https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf)

2. [贝叶斯个性化排序(BPR)算法小结](https://www.cnblogs.com/pinard/p/9128682.html)